## 필요한 데이터와 의존성 설치, 모듈 Importing


In [23]:
!kaggle competitions download - c kdtai-2
!unzip kdtai-2.zip - d dataset


zsh:1: command not found: kaggle
unzip:  cannot find or open kdtai-2.zip, kdtai-2.zip.zip or kdtai-2.zip.ZIP.


In [24]:
!pip list - -format = freeze


Package                       Version
----------------------------- ---------
appnope                       0.1.3
asttokens                     2.2.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
certifi                       2022.12.7
charset-normalizer            3.1.0
click                         8.1.3
comm                          0.1.3
cosine-annealing-warmup       2.0
debugpy                       1.6.7
decorator                     5.1.1
executing                     1.2.0
filelock                      3.12.0
future                        0.18.3
gensim                        4.3.1
idna                          3.4
importlib-metadata            6.6.0
ipykernel                     6.22.0
ipython                       8.12.0
jamotools                     0.1.10
jedi                          0.18.2
Jinja2                        3.1.2
joblib                        1.2.0
JPype1                        1.4.1
jupyter_client                8.2.0
jupyter_core    

In [25]:
%pip install - r requirements.txt


ERROR: Invalid requirement: '-'
Note: you may need to restart the kernel to use updated packages.


In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader, BatchSampler, random_split, WeightedRandomSampler, Subset
import pandas as pd
import os
import nltk
from konlpy.tag import Okt
import jamotools
import re
from gensim.models import FastText
from collections import Counter
from cosine_annealing_warmup import CosineAnnealingWarmupRestarts
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import gc
nltk.download('punkt')


[nltk_data] Downloading package punkt to /Users/lee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## FastText Model Training 및 저장


### Load Corpus


In [27]:
# okt = Okt()
# current_path = os.getcwd()
# train_df = pd.read_csv(os.path.join(current_path, "dataset", "train.csv"))
# train_corpus = []
# for sentence in train_df["text"]:
#     train_corpus.append(okt.morphs(sentence, stem=True))
# train_corpus[:3]


### Preprocess Corpus


In [28]:
# okt = Okt()
# train_corpus = okt.morphs(train_corpus, stem=True)
# train_corpus[:5]


In [29]:
# korean = re.compile('[^1!ㄱ-ㅣ가-힣]+')
# jamo_splited_corpus = []


# def jamo_split(word):
#     return korean.sub('', jamotools.split_syllables(word))


# for word in train_corpus:
#     jamo_splited_corpus.append(jamo_split(word))

# jamo_splited_corpus[:5]


### Train FastText Model


In [30]:
# model = FastText(train_corpus, vector_size=300,
#                  window=5, min_count=3, workers=4, sg=1)


In [31]:
# model.save("embedding_model/korean_fasttext.bin")


In [32]:
# device = torch.device("mps")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cpu


In [63]:
from enum import Enum
from nltk.tokenize import word_tokenize
from soynlp.hangle import decompose, character_is_korean
import re
from tqdm import tqdm
from konlpy.tag import Okt


class Dataset_type(Enum):
    TRAIN = 0
    TEST = 1


class Korean_dataset(Dataset):
    def __init__(self, file_path, dataset_type: Dataset_type,
                 embedding_model, use_jamo_split=False,
                 exclude_stopwords=False, max_length=None):
        super().__init__()
        self.file_path = file_path
        # self.transform = transform
        self.dataset_type = dataset_type
        self.data_df = pd.read_csv(self.file_path)
        self.embedding_model = embedding_model
        self.word2idx = embedding_model.wv.key_to_index
        self.use_jamo_split = use_jamo_split
        self.exclude_stopwords = exclude_stopwords
        self.max_length = max_length
        current_path = os.getcwd()
        stopwords_path = os.path.join(
            current_path, "utils", "StopWordKorean.csv")
        self.stopwords = pd.read_csv(stopwords_path, index_col=False)
        self.preprocessed_src_list = self._preprocessing(self.data_df)

    def __len__(self):
        return len(self.preprocessed_src_list)

    def __getitem__(self, idx):
        src = self.preprocessed_src_list[idx]
        if self.dataset_type == Dataset_type.TRAIN:
            trg = torch.tensor(self.data_df.loc[idx, "label"]).to(device)
            return src, trg
        else:
            return src

    def _preprocessing(self, df):
        src_list = []
        rows_to_drop = []
        okt = Okt()

        # 중복 제거 (Only Training)
        if self.dataset_type == Dataset_type.TRAIN:
            df.drop_duplicates(subset=['text'], inplace=True)

        # 한글, 영어, 공백, 숫자 빼고 전부 제거
        df["text"] = df["text"].apply(
            lambda x: re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\s]', '', x))

        # null 값 제거
        df.dropna(subset=["text"], inplace=True)
        df.reset_index(drop=True, inplace=True)

        for idx, text in tqdm(enumerate(df.loc[:, "text"])):
            # 형태소 분리
            tokens_per_sentence = okt.morphs(text, stem=True)

            # 불용어 제거 (Optional)
            if self.exclude_stopwords:
                tokens_per_sentence_ = self._remove_stopwords(
                    tokens_per_sentence)

                if len(tokens_per_sentence_) == 0:
                    if self.dataset_type == Dataset_type.TRAIN:
                        rows_to_drop.append(idx)
                        continue
                    # else:
                    #     src_list.append((torch.tensor(np.array(
                    #         [-1] * max_length)), torch.tensor(np.array([[-1] * 300 for _ in range(max_length)]))))
                    #     continue
                else:
                    tokens_per_sentence = tokens_per_sentence_

            # 자모 분리 (Optional)
            if self.use_jamo_split:
                tokens_per_sentence = self._split_jamo(tokens_per_sentence)

            # fastText 초기 model 기반 정수 인코딩
            encodings_of_tokens = self._indexing(
                tokens_per_sentence, self.max_length, self.embedding_model)

            # subwords 활용을 위한 초기 모델 기반 임베딩 벡터
            vectors_of_tokens = self._embedding(
                tokens_per_sentence, self.max_length, self.embedding_model)

            # src -> preprocessed_src_list[idx] =
            # ([encoding * max_vocab] , [[embeddingvector] * max_vocab]
            src = (encodings_of_tokens, vectors_of_tokens)
            src_list.append(src)

        if self.dataset_type == Dataset_type.TRAIN:
            df.drop(rows_to_drop, inplace=True)

        df.reset_index(drop=True, inplace=True)

        return src_list

    def _remove_stopwords(self, tokens_per_sentence):
        return [token for token in tokens_per_sentence
                if token not in self.stopwords["text"].values]

    def _split_jamo(self, tokens_per_sentence):
        korean = re.compile('[^1!ㄱ-ㅣ가-힣]+')
        return [korean.sub('', jamotools.split_syllables(
            token)) for token in tokens_per_sentence]

    def _indexing(self, tokens_per_sentence, max_length, embedding_model):
        padding_index = 0
        indexes_of_tokens = [padding_index] * max_length
        # 정수 인코딩 + padding
        for idx, token in enumerate(tokens_per_sentence):
            if idx == max_length:
                break
            if token in embedding_model.wv.key_to_index:
                token_id = self.word2idx[token]
            else:
                token_id = 0

            indexes_of_tokens[idx] = token_id
        return torch.tensor(np.array(indexes_of_tokens))

    def _embedding(self, tokens_per_sentence, max_length, embedding_model):
        padding_vector = embedding_model.wv['<pad>']
        vectors_of_tokens = [padding_vector] * max_length
        # 임베딩 + padding
        for idx, token in enumerate(tokens_per_sentence):
            if idx == max_length:
                break
            vectors_of_tokens[idx] = embedding_model.wv[token]
        return torch.tensor(np.array(vectors_of_tokens))


In [15]:
current_path = os.getcwd()
model_file_path = os.path.join(
    current_path, "embedding_model", "korean_fasttext.bin")
embedding_model = FastText.load(model_file_path)


In [64]:
import gc
gc.collect()
current_path = os.getcwd()
train_file_path = os.path.join(current_path, "dataset", "train.csv")
test_file_path = os.path.join(current_path, "dataset", "test.csv")

train_set = Korean_dataset(file_path=train_file_path,
                           dataset_type=Dataset_type.TRAIN,
                           use_jamo_split=False,
                           exclude_stopwords=False,
                           embedding_model=embedding_model,
                           max_length=20)
test_set = Korean_dataset(file_path=test_file_path,
                          dataset_type=Dataset_type.TEST,
                          use_jamo_split=False,
                          exclude_stopwords=False,
                          embedding_model=embedding_model,
                          max_length=20)


/var/folders/3_/19vyqy_n009677wdd2xcl_xh0000gn/T/ipykernel_4983/4113194596.py:31: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  self.stopwords = pd.read_csv(stopwords_path, index_col=False)
65334it [01:56, 562.40it/s]
/var/folders/3_/19vyqy_n009677wdd2xcl_xh0000gn/T/ipykernel_4983/4113194596.py:31: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  self.stopwords = pd.read_csv(stopwords_path, index_col=False)
13491it [00:26, 502.86it/s]


In [65]:
with open("dataset/trainset.pickle", "wb") as fw:
    pickle.dump(train_set, fw)


In [37]:
with open("dataset/trainset.pickle", "rb") as fr:
    train_set = pickle.load(fr)


In [38]:
train_set[0]


((tensor([9517, 1018,  282,  174,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0]),
  tensor([[-0.0094,  0.0447, -0.0309,  ...,  0.0317,  0.0436,  0.0030],
          [-0.0326,  0.1954, -0.3129,  ...,  0.0973, -0.0473, -0.0439],
          [-0.2051, -0.1395,  0.1422,  ...,  0.0236, -0.0012,  0.0843],
          ...,
          [-0.0004,  0.0006,  0.0003,  ..., -0.0005, -0.0005,  0.0009],
          [-0.0004,  0.0006,  0.0003,  ..., -0.0005, -0.0005,  0.0009],
          [-0.0004,  0.0006,  0.0003,  ..., -0.0005, -0.0005,  0.0009]])),
 tensor(1))

In [39]:
# class_sample_counts = [0] * 7
# for src, trg in train_set:
#     class_sample_counts[trg] += 1
# class_sample_counts


In [40]:
# total_count = sum(class_sample_counts)
# class_weights = [total_count /
#                  class_sample_count for class_sample_count in class_sample_counts]
# class_weights_tensor = torch.tensor(class_weights).to(device)
# class_weights_tensor


In [41]:
# samples_weight = torch.tensor(samples_weight)


In [42]:
print(len(train_set))


65288


In [102]:
from torch.nn.utils.rnn import pad_sequence


# class LSTM_Net(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim, embedding_model, max_length, num_layers=1, reverse=True):
#         super(LSTM_Net, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.num_layers = num_layers
#         self.embedding_model = embedding_model
#         self.word_vectors = self.embedding_model.wv
#         self.embedding = nn.Embedding.from_pretrained(
#             torch.from_numpy(self.word_vectors.vectors), padding_idx=0)
#         self.reverse = reverse
#         self.lstm = nn.LSTM(input_dim, hidden_dim,
#                             num_layers, batch_first=True, dropout=0.3)
#         self.fc1 = nn.Linear(2*input_dim, input_dim)
#         self.fc2 = nn.Linear((max_length + 1) * hidden_dim, hidden_dim)
#         self.fc3 = nn.Linear(hidden_dim, output_dim)

#     def forward(self, encoding, vector_1):
#         h0 = torch.zeros(self.num_layers, encoding.size(0),
#                          self.hidden_dim).to(encoding.device)
#         c0 = torch.zeros(self.num_layers, encoding.size(0),
#                          self.hidden_dim).to(encoding.device)
#         vector_1 = vector_1.flip(dims=[1]) if self.reverse else vector_1
#         encoding = encoding.flip(dims=[1]) if self.reverse else encoding
#         vector_2 = self.embedding(encoding)
#         x = torch.cat([vector_1, vector_2], dim=2)
#         x = self.fc1(x)
#         out, (hn, cn) = self.lstm(x, (h0, c0))
#         h_last = hn[num_layers-1]
#         score = (out @ h_last.unsqueeze(-1)).squeeze(-1)
#         a = nn.functional.softmax(score)
#         c = out * a.unsqueeze(-1)
#         h_tilde = torch.cat((c, h_last.unsqueeze(1)), dim=1)
#         h_tilde = nn.functional.tanh(
#             self.fc2(h_tilde.view(encoding.size(0), -1)))
#         x = self.fc3(h_tilde)
#         return x

class LSTM_Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, max_length, embedding_model, num_layers=1, reverse=True):
        super(LSTM_Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.embedding_model = embedding_model
        self.word_vectors = self.embedding_model.wv
        self.embedding = nn.Embedding.from_pretrained(
            torch.from_numpy(self.word_vectors.vectors), padding_idx=0)
        self.lstm = nn.LSTM(input_dim, hidden_dim,
                            num_layers, batch_first=True, dropout=0.3)
        self.fc1 = nn.Linear(2*input_dim, input_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.reverse = reverse

    def forward(self, encoding, vector_1):
        h0 = torch.zeros(self.num_layers, encoding.size(0),
                         self.hidden_dim).to(encoding.device)
        c0 = torch.zeros(self.num_layers, encoding.size(0),
                         self.hidden_dim).to(encoding
                                             .device)
        vector_1 = vector_1.flip(dims=[1]) if self.reverse else vector_1
        encoding = encoding.flip(dims=[1]) if self.reverse else encoding
        vector_2 = self.embedding(encoding)
        x = torch.cat([vector_1, vector_2], dim=2)
        x = self.fc1(x)
        out, (hn, cn) = self.lstm(x, (h0, c0))
        output = out[:, -1, :]
        x = self.fc2(output)
        return x


class CNN_1D_Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, embedding_model, max_length):
        super(CNN_1D_Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.embedding_model = embedding_model
        self.word_vectors = self.embedding_model.wv
        self.embedding = nn.Embedding.from_pretrained(
            torch.from_numpy(self.word_vectors.vectors), padding_idx=0)
        self.fc1 = nn.Linear(2*input_dim, input_dim)
        self.conv1 = nn.Conv1d(in_channels=input_dim,
                               out_channels=128, kernel_size=3)
        self.dropout = nn.Dropout(0.2)
        self.pool = nn.MaxPool1d(kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=32, kernel_size=4)
        self.fc2 = nn.Linear(32, output_dim)

    def forward(self, encoding, vector_1):
        h0 = torch.zeros(self.num_layers, encoding.size(0),
                         self.hidden_dim).to(encoding.device)
        c0 = torch.zeros(self.num_layers, encoding.size(0),
                         self.hidden_dim).to(encoding.device)
        # vector_1 = vector_1.flip(dims=[1]) if self.reverse else vector_1
        # encoding = encoding.flip(dims=[1]) if self.reverse else encoding
        vector_2 = self.embedding(encoding)
        x = torch.cat([vector_1, vector_2], dim=2)
        x = self.fc1(x)
        x = x.permute(0, 2, 1)  # batch_size x 128 x 32
        x = self.conv1(x)  # batch_size x 128 x 30
        x = nn.ReLU()(x)
        x = self.dropout(x)
        x = self.pool(x)  # batch_size x 128 x 10
        x = self.conv2(x)  # batch_size x 32 x 7
        x = nn.ReLU()(x)
        x = nn.MaxPool1d(kernel_size=x.size(2))(x)  # batch_size x 32 x 1
        x = x.view(x.size(0), -1)  # batch_size x 32
        x = self.fc2(x)  # batch_size x 1
        return x


input_dim = 300
hidden_dim = 128
output_dim = 7
num_layers = 4
batch_size = 64
num_of_epoch = 10
max_length = 20
learning_rate = 0.0005

# model = LSTM_Net(input_dim=input_dim, hidden_dim=hidden_dim,
#                  output_dim=output_dim, num_layers=num_layers, embedding_model=embedding_model, max_length=max_length).to(device)

model = CNN_1D_Net(input_dim=input_dim, hidden_dim=hidden_dim,
                   output_dim=output_dim, embedding_model=embedding_model, max_length=max_length)
# 손실 함수와 최적화 알고리즘을 정의합니다.
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = CosineAnnealingWarmupRestarts(
    optimizer, first_cycle_steps=4000, cycle_mult=1.0, max_lr=0.001, min_lr=0.0001, warmup_steps=500, gamma=1)
criterion = nn.CrossEntropyLoss().to(device)


def train(dataset, k_fold_split, batch_size, optimizer, scheduler, criterion, num_of_epoch, valid_random_seed=None, patience=3):
    # train_set_count = int(len(dataset) * valid_rate)
    # val_set_count = len(dataset) - train_set_count

    if valid_random_seed is not None:
        torch.manual_seed(valid_random_seed)

    # train_set, val_set = random_split(
    #     dataset, [train_set_count, val_set_count])

    # # sampler = WeightedRandomSampler(samples_weight.type(
    # #     'torch.DoubleTensor'), len(train_set), replacement=False)

    # train_loader = DataLoader(train_set, batch_size=batch_size,
    #                           shuffle=True)
    # val_loader = DataLoader(val_set, batch_size=batch_size,
    #                         shuffle=True)

    optimizer = optimizer
    criterion = nn.CrossEntropyLoss().to(device)

    skf = StratifiedKFold(n_splits=k_fold_split,
                          shuffle=True, random_state=valid_random_seed)

    labels = np.array(dataset.data_df.loc[:, "label"])

    for fold, (train_index, val_index) in enumerate(skf.split(np.zeros(len(dataset)), labels)):
        print("New Fold Start!!")
        print(f"Fold: {fold + 1}")

        if fold > 0:
            model.load_state_dict(torch.load('model/model_cnn.pt'))

        train_set = Subset(dataset, train_index)
        val_set = Subset(dataset, val_index)

        train_loader = DataLoader(
            train_set, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(
            val_set, batch_size=batch_size, shuffle=True)

        train_losses = []
        train_accs = []
        val_losses = []
        val_accs = []
        lrs = []
        preds = []
        targets = []
        is_early_stopping = False
        best_val_accuracy = 0
        best_f1_score = 0

        for epoch in tqdm(range(num_of_epoch)):
            model.train()
            correct_train = 0
            for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
                encoding = data[0]
                vector_1 = data[1]
                encoding, vector_1, target = encoding.to(
                    device), vector_1.to(device), target.to(device)
                optimizer.zero_grad()
                output = model(encoding, vector_1)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
                lrs.append(optimizer.param_groups[0]["lr"])
                scheduler.step()
                _, pred = torch.max(output.data, 1)
                preds.extend(pred.tolist())
                targets.extend(target.tolist())
                correct_train += pred.eq(target.view_as(pred)).sum().item()

                if batch_idx % 100 == 0:
                    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, lrs: {}'.format(
                        epoch + 1, batch_idx *
                        len(data[0]), len(train_loader.dataset),
                        100. * batch_idx / len(train_loader), loss.item(), lrs[-1]))

            train_accuracy = 100. * (correct_train / len(train_set))
            train_losses.append(loss.item())
            train_accs.append(train_accuracy)
            train_f1_score = f1_score(targets, preds, average="macro")
            print(correct_train)
            print(
                f"Epoch: {epoch + 1} - train accuracy: {train_accuracy}, train f1 score: {train_f1_score}")

            gc.collect()

            model.eval()
            preds = []
            targets = []
            correct_val = 0
            print(len(val_set))

            with torch.no_grad():
                for batch_idx, (data, target) in tqdm(enumerate(val_loader)):
                    encoding = data[0]
                    vector_1 = data[1]
                    encoding, vector_1, target = encoding.to(
                        device), vector_1.to(device), target.to(device)
                    output = model(encoding, vector_1)
                    loss = criterion(output, target)
                    _, pred = torch.max(output.data, 1)
                    preds.extend(pred.tolist())
                    targets.extend(target.tolist())
                    correct_val += pred.eq(target.view_as(pred)).sum().item()
                    # gc.collect()

                val_accuracy = 100. * (correct_val / len(val_set))
                val_losses.append(loss.item())
                val_accs.append(val_accuracy)
                val_f1_score = f1_score(targets, preds, average="macro")

                if val_f1_score > best_f1_score:
                    current_patience = patience
                    best_f1_score = val_f1_score
                    best_model = model.state_dict()
                    torch.save(best_model, 'model/model_cnn.pt')
                else:
                    current_patience -= 1
                    if current_patience < 0:
                        print("Early Stopping!")
                        is_early_stopping = True

                print('Epoch {} finished: train loss = {}, val loss = {}'.format(epoch + 1,
                                                                                 train_losses[-1], val_losses[-1]))
                print(
                    f"Epoch: {epoch + 1} - Validation accuracy: {val_accuracy} - Validation F1 Score: {val_f1_score}")

                if is_early_stopping:
                    break


기존 사전학습된 FaxtText 임베딩 모델


주어진 데이터로 만들어진 FastText 임베딩 모델


In [103]:
gc.collect()
train(dataset=train_set, k_fold_split=4, batch_size=batch_size, criterion=criterion,
      optimizer=optimizer, scheduler=scheduler, num_of_epoch=num_of_epoch, valid_random_seed=42)


New Fold Start!!
Fold: 1


  0%|          | 0/10 [00:00<?, ?it/s]

Train Epoch: 1 [0/49000 (0%)]	Loss: 1.964710, lrs: 0.0001


Train Epoch: 1 [6400/49000 (13%)]	Loss: 1.599041, lrs: 0.00028


Train Epoch: 1 [12800/49000 (26%)]	Loss: 1.344424, lrs: 0.00045999999999999996


...

Train Epoch: 5 [44800/49001 (91%)]	Loss: 0.730655, lrs: 0.0005911422095678922


766it [00:29, 26.29it/s]


38132
Epoch: 5 - train accuracy: 77.81882002408113, train f1 score: 0.7452656562201568
16333


256it [00:04, 56.74it/s]
 40%|████      | 4/10 [02:49<04:14, 42.49s/it]

Early Stopping!
Epoch 5 finished: train loss = 0.4442235827445984, val loss = 0.5677328109741211
Epoch: 5 - Validation accuracy: 78.19751423498438 - Validation F1 Score: 0.7525937940746786


In [104]:
model.load_state_dict(torch.load('model/model_cnn.pt'))


<All keys matched successfully>

In [105]:
def test(dataset, batch_size):

    test_loader = DataLoader(dataset, batch_size=batch_size,
                             shuffle=False)
    pred_list = []
    model.eval()
    with torch.no_grad():
        for data in tqdm(test_loader):
            encoding = data[0]
            vector_1 = data[1]
            encoding, vector_1 = encoding.to(
                device), vector_1.to(device)
            output = model(encoding, vector_1)
            _, pred = torch.max(output.data, 1)
            pred_list.extend(list(map(int, pred.cpu())))
    return pred_list


In [106]:
df_path = os.path.join("dataset", "submission.csv")
result = test(test_set, 64)


100%|██████████| 211/211 [00:03<00:00, 57.59it/s]


In [50]:
len(result)


13491

In [107]:
df = pd.read_csv(df_path)
df


,ID,label
0,0,NaN
1,1,NaN
2,2,NaN
3,3,NaN
4,4,NaN
...,...,...
13486,13486,NaN
13487,13487,NaN
13488,13488,NaN
13489,13489,NaN


In [108]:
for idx, pred in enumerate(result):
    df.loc[idx, "label"] = pred
df["label"] = df["label"].astype(int)
df.head(7)


,ID,label
0,0,4
1,1,4
2,2,3
3,3,6
4,4,2
5,5,0
6,6,4


In [109]:
df.to_csv("submission_result.csv", index=False)
